In [52]:
from lxml import html
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [59]:
page = requests.get('http://www.iucnredlist.org/details/12392')
tree = html.fromstring(page.content)
soup = bs(page.content, 'html.parser')

In [138]:
def get_population(page):
    for x in page.find('div',id='data_factsheet').find_all('td', class_='label'):
        if x.get_text()== 'Population:':
            return (x.parent.find_all('td')[-1].get_text())
        
def get_assessment(page):
    for x in page.find('div',id='data_factsheet').find_all('strong'):
        if x.get_text() == 'Justification:':
            return x.next_sibling.next_sibling

In [61]:
get_population(soup)

'Although elephant populations may at present be declining in parts of their range, major populations in Eastern and Southern Africa, accounting for over two thirds of all known elephants on the continent, have been surveyed, and are currently increasing at an average annual rate of 4.0% per annum (Blanc et al. 2005, 2007). As a result, more than 15,000 elephants are estimated to have been recruited into the population in 2006 and, if current rates of increase continue, the number of elephants born in these populations between 2005 and 2010 will be larger than the currently estimated total number of elephants in Central and West Africa combined. In other words, the magnitude of ongoing increases in Southern and Eastern Africa are likely to outweigh the magnitude of any likely declines in the other two regions.\n\n    For further information about this species,\n    see 12392_Loxodonta_africana.pdf.\n  \n\n    A PDF viewer such as\n    Adobe Reader\n    is required.\n  \n\n'

In [143]:
data = pd.read_excel('mammalis IUCN July 2018.xlsx')
for ID in data['Species ID']:
    page = requests.get('http://www.iucnredlist.org/details/' + str(ID))
    soup = bs(page.content, 'html.parser')
    data.loc[data['Species ID'] == ID,'Population data'] = get_population(soup)
    data.loc[data['Species ID'] == ID,'Assessment data'] = get_assessment(soup)
    

<Response [200]>

In [144]:
data['Population has numbers'] = ~pd.isnull(data['Population data'].str.extract(r'(\d+)'))


In [145]:
data['Population data length'] = data['Population data'].apply(lambda x: 0 if x is None else len(x))
data['Assessment data length'] = data['Assessment data'].apply(lambda x: 0 if x is None else len(x))

In [146]:
data.to_excel('IUCN_data_with_populations_and_assessment.xlsx')

In [148]:
pantheria_data = pd.read_csv('PanTHERIA_1-0_WR05_Aug2008.csv',sep='\t')
pantheria_data.columns

Index(['MSW05_Order', 'MSW05_Family', 'MSW05_Genus', 'MSW05_Species',
       'MSW05_Binomial', '1-1_ActivityCycle', '5-1_AdultBodyMass_g',
       '8-1_AdultForearmLen_mm', '13-1_AdultHeadBodyLen_mm',
       '2-1_AgeatEyeOpening_d', '3-1_AgeatFirstBirth_d',
       '18-1_BasalMetRate_mLO2hr', '5-2_BasalMetRateMass_g', '6-1_DietBreadth',
       '7-1_DispersalAge_d', '9-1_GestationLen_d', '12-1_HabitatBreadth',
       '22-1_HomeRange_km2', '22-2_HomeRange_Indiv_km2',
       '14-1_InterbirthInterval_d', '15-1_LitterSize', '16-1_LittersPerYear',
       '17-1_MaxLongevity_m', '5-3_NeonateBodyMass_g',
       '13-2_NeonateHeadBodyLen_mm', '21-1_PopulationDensity_n/km2',
       '10-1_PopulationGrpSize', '23-1_SexualMaturityAge_d',
       '10-2_SocialGrpSize', '24-1_TeatNumber', '12-2_Terrestriality',
       '6-2_TrophicLevel', '25-1_WeaningAge_d', '5-4_WeaningBodyMass_g',
       '13-3_WeaningHeadBodyLen_mm', 'References', '5-5_AdultBodyMass_g_EXT',
       '16-2_LittersPerYear_EXT', '5-6_NeonateB

In [150]:
tetra = pd.read_excel('tetraDensity.xls')
tetra.columns

Index(['Class', 'Order', 'Family', 'Genus', 'Species', 'Subspecies',
       'Longitude', 'Latitude', 'Locality', 'Country', 'Year', 'Season/Month',
       'Habitat', 'Sampling_Area', 'Sampling_Area_unit', 'Density',
       'Density_unit', 'Sampling_Method', 'Method_info', 'Notes', 'Ref_N'],
      dtype='object')